# HW3 Q4 [10 pts]



## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> add any cells to this Jupyter Notebook, because that will crash the autograder.
</div>


All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  



<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`load_data()`

`exclude_no_pickuplocations()`

`exclude_no_tripdistance()`

`include_fare_range()`

`get_highest_tip()`

`get_total_toll()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from NYC Taxi Trips during auto-grading. You do not need to write code for unreasonable scenarios.  

Since the overall correctness of your code will require multiple function to work together correctly (i.e., all methods are interdepedent), implementing only a subset of the functions likely will lead to a low score.

### Helper functions

You are permitted to write additional helper functions, or use additional instance variables so long as the previously described functions work as required.  

#### Pyspark Imports
<span style="color:red">*Please don't modify the below cell*</span>

In [37]:
import pyspark
from pyspark.sql import SQLContext

#### Define Spark Context
<span style="color:red">*Please don't modify the below cell*</span>

In [38]:
sc
sqlContext = SQLContext(sc)

### Student Section - Please compete all the functions below

#### Function to return GT Username

In [39]:
def user():
        """
        :return: string
        your GTUsername, NOT your 9-Digit GTId  
        """         
        return 'xxue35'

#### Function to load data

In [40]:
def load_data(gcp_storage_path):
    """
        :param gcp_storage_path: string (full gs path including file name e.g gs://bucket_name/data.csv) 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to load yellow_tripdata_2019-01.csv data from your GCP storage bucket#
    #                                                              #        
    ################################################################
    df=spark.read.csv(gcp_storage_path,header=True)
    df=df.withColumn('trip_distance',df['trip_distance'].cast('float'))
    df=df.withColumn('fare_amount',df['fare_amount'].cast('float'))
    df=df.withColumn('tip_amount',df['tip_amount'].cast('float'))
    df=df.withColumn('tolls_amount',df['tolls_amount'].cast('float'))
        
    return df

#### Function to exclude trips that don't have a pickup location

In [41]:
def exclude_no_pickuplocations(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to exclude trips with no pickup locations               #
    #                                                              #        
    ################################################################
    df=df.na.drop(subset=["PULocationID"])
    
    return df

#### Function to exclude trips with no distance

In [42]:
def exclude_no_tripdistance(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to exclude trips with no trip distances                 #
    #                                                              #        
    ################################################################
    df=df.na.drop(subset=["trip_distance"])
    df=df.filter((df.trip_distance !=0))
    
    return df

#### Function to include fare amount between the range of 20 to 60 Dollars

In [44]:
def include_fare_range(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to include trips with only within the fare range of     #
    # 20 to 60 dollars                                             #        
    ################################################################
    df=df.filter((df.fare_amount>=20)&(df.fare_amount<=60))
    return df

#### Function to get the highest tip amount

In [46]:
def get_highest_tip(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: float (rounded to 2 digits)  
    """
    
    ################################################################
    # code to get the highest tip                                  #
    #                                                              #        
    ################################################################
    df.createOrReplaceTempView("taxi")
    outdf=sqlContext.sql("""select round(max(tip_amount),2) as tip from taxi
                              
                        """).collect()
    max_tip=outdf[0].tip
    return round(max_tip,2)

#### Function to get total toll amount

In [48]:
def get_total_toll(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: float (rounded to 2 digits)  
    """
    
    ################################################################
    # code to get total toll amount                                #
    #                                                              #        
    ################################################################
    df.createOrReplaceTempView("taxi")
    outdf=sqlContext.sql("""select sum(tolls_amount) as toll from taxi
                              
                        """).collect()
    total_toll=outdf[0].toll
    return round(total_toll,2)

### Run above functions and print

#### Uncomment the cells below and test your implemented functions

#### Load data from yellow_tripdata_2019-01.csv 

In [50]:
gcp_storage_path = "gs://xxue35/yellow_tripdata_2019-01.csv"
df = load_data(gcp_storage_path)
df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)



#### Print total numbers of rows in the dataframe

In [51]:
df.count()

7667792

#### Print total number of rows in the dataframe after excluding trips with no pickup location

In [52]:
df_no_pickup_locations = exclude_no_pickuplocations(df)
df_no_pickup_locations.count()
#before:7667792
#after: 7667792

7667792

#### Print total number of rows in the dataframe after exclude trips with no distance

In [53]:
df_no_trip_distance = exclude_no_tripdistance(df_no_pickup_locations)
df_no_trip_distance.count()


7613022

#### Print total number of rows in the dataframe after including trips with fair amount between the range of 20 to 60 Dollars

In [54]:
df_include_fare_range = include_fare_range(df_no_trip_distance)
df_include_fare_range.count()


957105

#### Print the highest tip amount

In [55]:
max_tip = get_highest_tip(df_include_fare_range)
print(max_tip)

444.8


#### Print the total toll amount

In [56]:
total_toll = get_total_toll(df_include_fare_range)
print(total_toll)

2092255.51
